### GPT-3 Test and Learn
Copied from https://blog.dspaces.io/2021/04/15/a-test-and-learn-approach-to-developing-with-openais-gpt-3/
I only changed it to work with a CSV instead of Google BigQuery

In [5]:
# Import Modules
import os
import openai
import pandas as pd
import datetime
from dotenv import load_dotenv

# OpenAI Key
load_dotenv()
OPENAI_KEY = os.environ.get('OPENAI_KEY')
openai.api_key = OPENAI_KEY

In [15]:
# Name your project and define a goal
project_name = "openai_test_and_learn"
project_goal = "Build a test and learn framework around developing on the OpenAI GPT-3 API"
project_details = {
    "project_name": project_name,
    "project_goal": project_goal
}

In [6]:
def build_prompt(user_input):
    """
    Background: This function takes the parameter user_input and incorporates it into the prompt.
    
    Params:
        - user_input (str): User generated input that we will pass into the prompt. Ex: 'Once upon a time'
    
    Returns: 
        - prompt (str): The prompt that includes user_input and will be sent along with the GPT-3 request.
    
    """

    # The first line describes what we want GPT-3 to do. 
    prompt = f"""Create a story about exploring the moon.
    
{user_input}"""
    
    return prompt

In [16]:
def run_openai_experiment(params, project_details):
    
    """
    Background: This function captures all details about a request to the OpenAI API and saves this data to CSV.
    
    Params:
        - params (dict): Contains all of the parameters used in an OpenAI GPT-3 request
     
    Returns:
        df: A dataframe with details about the request + response
    
    """
    
    # Create a Timestamp
    timestamp = datetime.datetime.now()
    
    # Print Prompt
    print('*** PROMPT ***')
    print(params['prompt'])
    
    # Make a request to OpenAI using the selected params
    response = openai.Completion.create(
      engine=params['engine'],
      prompt=params['prompt'],
      max_tokens=params['max_tokens'],
      temperature=params['temperature'],
      top_p=params['top_p'],
      frequency_penalty=params['frequency_penalty'],
      presence_penalty=params['presence_penalty'],  
      n=params['n'],
      stream = params['stream'],
      logprobs=params['logprobs'],
      stop = params['stop']
    )
    
    # Parse out the response   
    response_text = response.choices[0]['text']
    response_id = response['id']
    response_model = response['model']
    response_object = response['object']
            
    # Display the output to the user
    print('*** RESPONSE FROM OPENAI ***')
    print(response_text)
    
    # Prompt satisfaction question
    satisfaction_score = input('Please rate the quality of this output from 1 to 10, where 1 is not at all satisfied and 10 is extremely satisfied:\n ')
    
    # Record observations about the output
    observations = input("Please note any observations about the output - eg. how to improve / changes made / etc:\n ")
    
    # Gather all of our data about the initial request, response and our observations
    data = {
        'timestamp':timestamp,
        'project_name': project_details['project_name'],
        'project_goal': project_details['project_goal'],
        'prompt': params['prompt'],        
        'response_text': response_text,
        'satisfaction_score':satisfaction_score,
        'observations':observations,
        'response_id': response_id,
        'response_model': response_model,
        'response_object': response_object,
        'response_length': len(response_text),
        'prompt_length': len(params['prompt']),
        'engine':params['engine'],      
        'max_tokens':params['max_tokens'],
        'temperature':params['temperature'],
        'top_p':params['top_p'],
        'frequency_penalty':params['frequency_penalty'],
        'presence_penalty':params['presence_penalty'],        
        'n':params['n'],
        'stream':params['stream'],
        'logprobs':params['logprobs'],
        'stop':params['stop']
    }    
    
    # Create a dataframe based on data    
    df = pd.DataFrame([data])
    
    # Send data to CSV, merge if db exists
    db_name = "gpt_responses.csv"
    if os.path.isfile(db_name):
        db = pd.read_csv(db_name)
        df = db.append(df)
        df.to_csv(db_name)
        
    else:
        df.to_csv(db_name)
        
    print(f"The following data successfully saved to CSV '{db_name}':")
    
    return df.tail(3)

In [18]:
# The opening line to our story we want GPT-3 to build on.
user_input = 'Once upon a time,'

# Let's take user_input and use it to generate a prompt
my_prompt = build_prompt(user_input)

# GPT-3 Request parameters
model_params = {
    'engine': 'davinci',
    'prompt': my_prompt,
    'max_tokens': 50,
    'temperature': 1.0,
    'top_p': 0.5,
    'frequency_penalty': 0,
    'presence_penalty': 0,
    'n': 1,
    'stream': None,
    'logprobs': None,
    'stop':None
}

# Pass in GPT-3 model parameters and project details. 
run_openai_experiment(model_params, project_details)

*** PROMPT ***
Create a story about exploring the moon.
    
Once upon a time,
*** RESPONSE FROM OPENAI ***
 there was a man who wanted to go to the moon. He wanted to go to the moon because he wanted to see what it was like. He wanted to see if it was really as beautiful as people said it was. He wanted to see if


Please rate the quality of this output from 1 to 10, where 1 is not at all satisfied and 10 is extremely satisfied:
  9
Please note any observations about the output - eg. how to improve / changes made / etc:
  


The following data successfully saved to CSV 'gpt_responses.csv':


,Unnamed: 0,timestamp,project_name,project_goal,prompt,response_text,satisfaction_score,observations,response_id,response_model,...,engine,max_tokens,temperature,top_p,frequency_penalty,presence_penalty,n,stream,logprobs,stop
0,0.0,2021-04-22 18:19:24.287694,openai_test_and_learn,Build a test and learn framework around develo...,Create a story about exploring the moon.\n ...,there was a family who had a son. He was the ...,8,didnt mention the moon,cmpl-2rXH3E6nWdWsvCvljH0NQ1ESeVFjd,davinci:2020-05-03,...,davinci,50,1.0,0.5,0,0,1,NaN,NaN,NaN
0,NaN,2021-04-22 18:20:00.973071,openai_test_and_learn,Build a test and learn framework around develo...,Create a story about exploring the moon.\n ...,there was a man who wanted to go to the moon....,9,,cmpl-2rXHdnpmLPHLA5ikXO30Qz9EEkoNn,davinci:2020-05-03,...,davinci,50,1.0,0.5,0,0,1,None,None,None
